<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/Parseo_de_OrdenesdelDia_ListadoCerrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "Scraps PDFs.csv"
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "base_formateada_senado_snowball.csv"

Mounted at /content/drive
 96% 225M/234M [00:02<00:00, 90.3MB/s]
100% 234M/234M [00:02<00:00, 101MB/s] 
 90% 129M/143M [00:02<00:00, 43.3MB/s]
100% 143M/143M [00:02<00:00, 54.3MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
# Cargamos la base
base = pd.read_csv('/content/Scraps PDFs.csv')
base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')

<ipython-input-3-e0eeee1141a8>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')


In [4]:
base_parseada.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens'],
      dtype='object')

In [144]:
import re

def generar(l):
  df=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'orden'])
  fila=base['link'][base['link'] == l[6]].index[0]
  texto=str(base.loc[fila].text)
  discurso=str(l[3])


  #ordenes = texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')].split('\n\n')
  ordenes=['ACTAS', 'REGLAMENTO DEL SENADO', 'INCOMPATIBILIDADES', 'COMISIONES MIXTAS', 'CONVENIOS Y ACUERDOS DE COOPERACIÓN ENTRE COMUNIDADES AUTÓNOMAS']

  o=0
  for orden in ordenes:
    print(texto[texto.find('SUMARIO'):].find(discurso.replace('\n','')))
    print(texto[texto.find('SUMARIO'):].find(orden.upper()))
    print('----------')
    if texto[texto.find('SUMARIO'):].find(discurso)>texto[texto.find('SUMARIO'):].find(orden.upper()): o+=1

  print(o)
  df['Nombre/Cargo']=[l[2]]
  df['Discurso']=[l[3]]
  df['Cargo']=[l[4]]
  df['Nombre']=[l[5]]
  df['Link']=[l[6]]
  df['Legislatura']=[l[7]]
  df['tipo']=[l[8]]
  df['fecha']=[l[9]]
  df['fecha_formato']=[l[10]]
  df['nombre']=[l[11]]
  df['apellidos']=[l[12]]
  df['procedLiteral']=[l[13]]
  df['procedLugar']=[l[14]]
  df['grupoNombre']=[l[15]]
  df['tokens']=[l[16]]
  try:
    df['orden']=[ordenes[o-1]]
  except:
    df['orden']=[""]


  return df

In [145]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'orden'])

In [146]:
for row in base_parseada.values[26:27]:
  base2 = pd.concat([base2, generar(row)], axis=0)

7659
984
----------
7659
1220
----------
7659
3523
----------
7659
4850
----------
7659
5355
----------
5


In [147]:
base2

,Nombre/Cargo,Discurso,Cargo,Nombre,Link,Legislatura,tipo,fecha,fecha_formato,nombre,apellidos,procedLiteral,procedLugar,grupoNombre,tokens,orden
0,El señor OSÉS RAMÍREZ,"Sí, juro.",OSÉS RAMÍREZ,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,DANIEL,OSÉS RAMÍREZ,Electo,Electo: La Rioja,GPP,['jur'],CONVENIOS Y ACUERDOS DE COOPERACIÓN ENTRE COMU...


In [ ]:
base2=base2.reset_index(drop=True)

In [ ]:
base2['fecha_formato'] = pd.to_datetime(base2['fecha_formato'], errors='coerce')

In [ ]:
from google.colab import files
base2.to_csv('base_formateada_senado_snowball_conorden.csv', escapechar='\\')
files.download('base_formateada_senado_snowball_conorden.csv')